In [ ]:
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [40]:
graph = Neo4jGraph(
    url="bolt://localhost:7689",
    username="neo4j",
    password="admin_neo4j"
)

In [111]:
movies_data = {
    "Rajkumar Hirani": [
        ("3 Idiots", ["Aamir Khan", "R. Madhavan"], 8.4),
        ("PK", ["Aamir Khan", "Anushka Sharma"], 8.2),
        ("Sanju", ["Ranbir Kapoor", "Paresh Rawal"], 7.6),
        ("Lage Raho Munna Bhai", ["Sanjay Dutt", "Arshad Warsi"], 8.0),
    ],
    "Sanjay Leela Bhansali": [
        ("Padmaavat", ["Ranveer Singh", "Deepika Padukone"], 7.0),
        ("Bajirao Mastani", ["Ranveer Singh", "Priyanka Chopra"], 7.2),
        ("Black", ["Amitabh Bachchan", "Rani Mukerji"], 8.2),
        ("Devdas", ["Shah Rukh Khan", "Aishwarya Rai"], 7.5),
    ],
    "Anurag Kashyap": [
        ("Gangs of Wasseypur", ["Nawazuddin Siddiqui", "Manoj Bajpayee"], 8.2),
        ("Ugly", ["Ronit Roy", "Rahul Bhat"], 8.0),
        ("Raman Raghav 2.0", ["Nawazuddin Siddiqui", "Vicky Kaushal"], 7.3),
        ("Dev.D", ["Abhay Deol", "Mahie Gill"], 7.9),
    ],
    "Zoya Akhtar": [
        ("Zindagi Na Milegi Dobara", ["Hrithik Roshan", "Farhan Akhtar"], 8.1),
        ("Dil Dhadakne Do", ["Ranveer Singh", "Priyanka Chopra"], 6.8),
        ("Gully Boy", ["Ranveer Singh", "Alia Bhatt"], 7.9),
        ("Luck by Chance", ["Farhan Akhtar", "Konkona Sen Sharma"], 7.0),
    ],
    "Karan Johar": [
        ("Kabhi Khushi Kabhie Gham", ["Shah Rukh Khan", "Kajol"], 7.4),
        ("My Name Is Khan", ["Shah Rukh Khan", "Kajol"], 7.9),
        ("Student of the Year", ["Sidharth Malhotra", "Alia Bhatt"], 5.6),
        ("Ae Dil Hai Mushkil", ["Ranbir Kapoor", "Anushka Sharma"], 5.8),
    ]
}


In [113]:
# Populate Neo4j database with movie data
for director, movies in movies_data.items():
    for title, actors, rating in movies:
        graph.query("""
            MERGE (d:Director {name: $director})
            MERGE (m:Movie {title: $title})
            SET m.rating = $rating
            MERGE (d)-[:DIRECTED]->(m)
        """, {"director": director, "title": title, "rating": rating})

        for actor in actors:
            graph.query("""
                MERGE (a:Actor {name: $actor})
                MERGE (m:Movie {title: $title})
                MERGE (a)-[:ACTED_IN]->(m)
            """, {"actor": actor, "title": title})

In [114]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [119]:
SCHEMA = """
Node types:
- Movie(title: string, released: int, genre: string, rating: float)
- Actor(name: string)
- Director(name: string)

Relationships:
- (Actor)-[:ACTED_IN]->(Movie)
- (Director)-[:DIRECTED]->(Movie)
"""

cypher_prompt = ChatPromptTemplate.from_template("""
You are an expert Cypher generator for a Neo4j database. Based on the schema below and the user question, write a Cypher query.

Schema:
{schema}

Question:
{question}

Cypher Query:
""")

def movie_context(query: str) -> str:
    cypher_generator = (
        {"schema": RunnablePassthrough(), "question": RunnablePassthrough()}
        | cypher_prompt
        | llm
        | StrOutputParser()
    )
    cypher = cypher_generator.invoke({"schema": SCHEMA, "question": query})

    result = graph.query(cypher)
    
    return str(result)

In [120]:
prompt = ChatPromptTemplate.from_template("""
You are a movie expert chatbot. Use the following knowledge from a graph to answer the question. 
Only include movies from the graph context.

Graph Context:
{context}

Question:
{question}
""")

chain = (
    {"context": movie_context, "question": RunnablePassthrough()}
    | prompt
    | llm
)


In [121]:
query = "list some actors who acted in movies directed by Rajkumar Hirani. name the movies as well."
response = chain.invoke(query)
print(response.content)

Actors who acted in movies directed by Rajkumar Hirani are:

1. Aamir Khan in "3 Idiots" and "PK"
2. R. Madhavan in "3 Idiots"
3. Anushka Sharma in "PK"


In [122]:
query = "list some movies directed by Rajkumar Hirani and acted by Shah Rukh Khan."
response = chain.invoke(query)
print(response.content)

I'm sorry, but there are no movies directed by Rajkumar Hirani and acted by Shah Rukh Khan in the graph context provided.


In [123]:
query = "list some movies acted by Shah Rukh Khan. Name the directors as well."
response = chain.invoke(query)
print(response.content)

Some movies acted by Shah Rukh Khan from the graph context are:
1. Devdas directed by Sanjay Leela Bhansali
2. My Name Is Khan directed by Karan Johar


In [124]:
query = "list some movies along with ratings"
response = chain.invoke(query)
print(response.content)

- 3 Idiots: 8.4
- PK: 8.2
- Sanju: 7.6
- Lage Raho Munna Bhai: 8.0
- Black: 8.2
- Gangs of Wasseypur: 8.2
- Ugly: 8.0
- Dev.D: 7.9
- Zindagi Na Milegi Dobara: 8.1
- Gully Boy: 7.9


In [125]:
query = "list some movies having rating greater than 8.0"
response = chain.invoke(query)
print(response.content)

Sure! Some movies from the graph context with a rating greater than 8.0 are:
1. 3 Idiots (8.4)
2. PK (8.2)
3. Black (8.2)
4. Gangs of Wasseypur (8.2)
